In [ ]:
# PIPELINE COMPLETO: PREDICCIÓN DE CONVERSIÓN DE LEADS
# Curso: Data Science y su relación con la Inteligencia Artificial
# Etapa 1: Desde la carga de datos hasta la evaluación del modelo

"""
CONTEXTO DEL NEGOCIO:
Somos el equipo de marketing de una empresa edtech que recibe miles de formularios
de contacto semanalmente. El equipo comercial está sobrepasado y necesitamos
priorizar los leads con mayor probabilidad de conversión.

OBJETIVO:
Construir un modelo de Machine Learning que prediga qué leads tienen mayor
probabilidad de convertirse en clientes, para que el equipo comercial pueda
enfocarse en ellos primero.
"""

# ============================================================================
# SECCIÓN 1: CONFIGURACIÓN INICIAL
# ============================================================================

# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pickle
import requests
import io
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

# Configuramos el estilo de las visualizaciones
plt.style.use('fivethirtyeight')
sns.set(style='whitegrid')

print("🚀 Iniciando el pipeline de Machine Learning para la predicción de conversión de leads")

In [ ]:
# ============================================================================
# SECCIÓN 2: CARGA DE DATOS
# ============================================================================

# Cargamos el dataset directamente desde una URL o desde el sistema local
try:
    # Intentamos cargar desde URL (probablemente fallará, pero lo dejamos como opción)
    url = "https://raw.githubusercontent.com/usuario/repositorio/main/Lead_Conversion.csv"
    df = pd.read_csv(url)
    print(f"✅ Datos cargados exitosamente desde URL. Dimensiones: {df.shape}")
except:
    print("❌ Error al cargar desde URL. Solicitando carga manual...")
    # Alternativa: Carga manual desde Colab
    uploaded = files.upload()
    # Obtenemos el nombre del archivo subido (primera clave en el diccionario)
    file_key = list(uploaded.keys())[0]
    df = pd.read_csv(io.BytesIO(uploaded[file_key]))
    print(f"✅ Datos cargados exitosamente de forma manual. Dimensiones: {df.shape}")

# Mostramos las primeras filas para verificar
print("\nPrimeras filas del dataset:")
print(df.head())

In [ ]:
# ============================================================================
# SECCIÓN 3: EXPLORACIÓN Y ANÁLISIS DE DATOS (EDA)
# ============================================================================

print("\n📊 EXPLORANDO LOS DATOS")

# Información general del dataset
print("\nInformación del dataset:")
print(df.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df.describe())

# Verificamos valores nulos
print("\nVerificando valores nulos:")
print(df.isnull().sum())

# Definimos la variable objetivo
target_column = 'Converted'
print(f"\nVariable objetivo: {target_column}")
print(f"Valores únicos: {df[target_column].unique()}")
print(f"Distribución:\n{df[target_column].value_counts()}")

# Visualizamos la distribución de la variable objetivo
plt.figure(figsize=(8, 8))
df[target_column].value_counts().plot.pie(autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
plt.title('Distribución de Leads por Conversión')
plt.ylabel('')
plt.show()

In [ ]:
# ============================================================================
# SECCIÓN 4: PREPARACIÓN DE DATOS PARA ANÁLISIS
# ============================================================================

# Separamos características (X) y variable objetivo (y)
X = df.drop(target_column, axis=1)
y_original = df[target_column]

# Si la variable objetivo es categórica (objeto), la convertimos a numérica
if y_original.dtype == 'object':
    print("\nConvirtiendo variable objetivo a formato numérico...")
    # Asumimos que "Yes" es el valor positivo (1) y "No" es el valor negativo (0)
    label_map = {"Yes": 1, "No": 0}
    y = y_original.map(label_map)
    print(f"Mapeo: {label_map}")
    print(f"Distribución después de la conversión:\n{y.value_counts()}")
else:
    y = y_original

# Identificamos tipos de columnas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nCaracterísticas numéricas ({len(numeric_features)}): {numeric_features}")
print(f"Características categóricas ({len(categorical_features)}): {categorical_features}")

In [ ]:
# ============================================================================
# SECCIÓN 5: ANÁLISIS DE RELACIONES ENTRE VARIABLES
# ============================================================================

print("\n📊 ANALIZANDO RELACIONES ENTRE VARIABLES")

# Matriz de correlación para variables numéricas
print("\n5.1. Matriz de correlación entre variables numéricas:")
plt.figure(figsize=(12, 10))
# Calculamos la matriz de correlación solo para variables numéricas
correlation_matrix = X[numeric_features].corr()

# Creamos un heatmap con anotaciones
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix,
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            mask=mask,
            linewidths=0.5,
            vmin=-1,
            vmax=1,
            center=0,
            square=True)
plt.title('Matriz de Correlación entre Variables Numéricas', fontsize=14)
plt.tight_layout()
plt.show()

# También mostramos correlación con la variable objetivo
# Creamos un dataframe temporal con variables numéricas y la variable objetivo
corr_with_target = X[numeric_features].copy()
corr_with_target['target'] = y
correlation_with_target = corr_with_target.corr()['target'].drop('target').sort_values(ascending=False)

plt.figure(figsize=(10, 6))
correlation_with_target.plot(kind='bar', color='teal')
plt.title('Correlación de Variables Numéricas con la Conversión', fontsize=14)
plt.xlabel('Variables')
plt.ylabel('Coeficiente de Correlación')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\nCorrelación con la variable objetivo:")
for variable, corr in correlation_with_target.items():
    print(f"  - {variable}: {corr:.4f}")

# Analizamos relaciones entre algunas características y la conversión
print("\n5.2. Analizando relaciones entre características y conversión...")

# Analizamos variables numéricas (seleccionamos hasta 3)
for col in numeric_features[:3]:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=y_original, y=X[col])
    plt.title(f'{col} por Estado de Conversión')
    plt.ylabel(col)
    plt.xlabel('Convertido')
    plt.show()

# Analizamos variables categóricas (seleccionamos hasta 2)
for col in categorical_features[:2]:
    if col != 'Prospect_ID':  # Excluimos el ID que no aporta valor al análisis
        plt.figure(figsize=(12, 6))
        cross_tab = pd.crosstab(X[col], y_original)
        cross_tab_pct = cross_tab.div(cross_tab.sum(axis=1), axis=0)
        cross_tab_pct.plot(kind='bar', stacked=True)
        plt.title(f'Tasa de Conversión por {col}')
        plt.xlabel(col)
        plt.ylabel('Porcentaje')
        plt.legend(title='Convertido')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
# ============================================================================
# SECCIÓN 6: MANEJO DE VALORES NULOS Y DIVISIÓN DE DATOS
# ============================================================================

# Manejamos valores nulos
print("\nManejando valores nulos...")
# Imputamos valores para variables numéricas y categóricas
for col in X.columns:
    if X[col].isnull().sum() > 0:
        if col in numeric_features:
            X[col].fillna(X[col].median(), inplace=True)
            print(f"  - Imputados valores en {col} con la mediana")
        else:
            X[col].fillna(X[col].mode()[0], inplace=True)
            print(f"  - Imputados valores en {col} con la moda")

# Dividimos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nConjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}")

# Preparamos el preprocesamiento para el pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Excluimos el ID de cliente para evitar overfitting
if 'Prospect_ID' in categorical_features:
    categorical_features.remove('Prospect_ID')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("\n✅ Preprocesamiento configurado")

In [ ]:
# ============================================================================
# SECCIÓN 7: ENTRENAMIENTO DEL MODELO
# ============================================================================
print("\n🧠 ENTRENANDO EL MODELO")

# Configuramos el pipeline completo con RandomForest
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Definimos los hiperparámetros para búsqueda
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}

# Configuramos Grid Search para encontrar los mejores hiperparámetros
print("\nBuscando los mejores hiperparámetros (esto puede tomar unos minutos)...")
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

# Entrenamos el modelo
grid_search.fit(X_train, y_train)

print(f"\nMejores hiperparámetros encontrados: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


In [ ]:
# ============================================================================
# SECCIÓN 8: EVALUACIÓN DEL MODELO
# ============================================================================

print("\n📊 EVALUANDO EL MODELO")

# Hacemos predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculamos métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Mostramos el reporte de clasificación detallado
print("\nReporte de clasificación completo:")
print(classification_report(y_test, y_pred))

# Visualizamos la matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.xticks([0.5, 1.5], ['No Convertido', 'Convertido'])
plt.yticks([0.5, 1.5], ['No Convertido', 'Convertido'])
plt.show()

# Visualizamos la curva ROC
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Área ROC = {roc_auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# SECCIÓN 9: ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS
# ============================================================================
print("\n🔍 ANALIZANDO IMPORTANCIA DE CARACTERÍSTICAS")

# Método simplificado para mostrar claramente la importancia de características con nombres reales
try:
    # Creamos un modelo específico para interpretación usando los datos originales
    from sklearn.ensemble import RandomForestClassifier

    # Necesitamos características en formato numérico para este análisis
    X_interpret = X.copy()

    # Convertimos categóricas a numéricas usando LabelEncoder para cada columna
    for col in categorical_features:
        if col != 'Prospect_ID':  # Excluimos el ID
            le = LabelEncoder()
            # Si hay valores nulos, primero los llenamos
            if X_interpret[col].isnull().sum() > 0:
                X_interpret[col] = X_interpret[col].fillna(X_interpret[col].mode()[0])
            X_interpret[col] = le.fit_transform(X_interpret[col])

    # Entrenamos un modelo simple directamente en los datos transformados
    interpret_model = RandomForestClassifier(
        n_estimators=100,
        random_state=42
    )
    interpret_model.fit(X_interpret, y)

    # Obtenemos la importancia de características
    importances = interpret_model.feature_importances_

    # Creamos un DataFrame para visualización con nombres reales
    feature_importance = pd.DataFrame({
        'Feature': X_interpret.columns,
        'Importance': importances
    }).sort_values('Importance', ascending=False)


    # Tomamos las 10 características más importantes
    top_n = 10
    top_features = feature_importance.head(top_n)

    # Creamos un gráfico más impactante y claro
    plt.figure(figsize=(14, 10))

    # Invertimos el orden para que la más importante esté arriba
    ax = sns.barplot(
        x='Importance',
        y='Feature',
        data=top_features.iloc[::-1],
        palette='viridis'
    )

    # Personalizamos el gráfico para mayor claridad
    plt.title('Las 10 Variables Más Importantes para Predecir la Conversión de Leads', fontsize=16)
    plt.xlabel('Importancia Relativa (%)', fontsize=14)
    plt.ylabel('Variable', fontsize=14)

    # Agregamos cuadrícula horizontal para facilitar la lectura
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Añadimos etiquetas con los valores exactos
    for i, v in enumerate(top_features['Importance'].iloc[::-1]):
        ax.text(v + 0.01, i, f'{v:.2%}', va='center', fontsize=12)

    # Ajustamos las etiquetas del eje y para mayor claridad
    plt.yticks(fontsize=12)

    # Formateamos el eje x como porcentaje
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{x:.0%}"))

    plt.tight_layout()
    plt.show()

    # Mostramos también una tabla detallada con las variables más importantes y sus valores
    print("\nImportancia relativa de las variables (ordenadas de mayor a menor):")
    importance_table = feature_importance.copy()

    # Agregamos columnas con importancia en formato porcentual y en orden decreciente
    importance_table['Importancia (%)'] = importance_table['Importance'].apply(lambda x: f"{x:.2%}")
    importance_table['Orden'] = range(1, len(importance_table) + 1)

    # Mostramos la tabla con formato
    print(importance_table[['Orden', 'Feature', 'Importancia (%)']].to_string(index=False))

except Exception as e:
    print(f"Error al analizar importancia de características: {e}")

    # Plan alternativo más simple si falla el método principal
    print("\nUtilizando método alternativo para mostrar la importancia de variables...")

    # Analizamos correlaciones para variables numéricas
    numeric_df = X[numeric_features].copy()
    numeric_df['target'] = y

    # Calculamos correlaciones absolutas (importa la magnitud, no la dirección)
    correlations = numeric_df.corr()['target'].abs().sort_values(ascending=False)

    # Creamos un único gráfico claro
    plt.figure(figsize=(12, 8))
    correlations.drop('target').head(10).sort_values().plot(
        kind='barh',
        color='teal'
    )
    plt.title('Variables Numéricas con Mayor Impacto en la Conversión de Leads', fontsize=16)
    plt.xlabel('Magnitud de Correlación', fontsize=14)
    plt.ylabel('Variable', fontsize=14)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Mostramos tabla con correlaciones
    corr_table = pd.DataFrame({
        'Variable': correlations.drop('target').index,
        'Correlación': correlations.drop('target').values,
        'Correlación (%)': correlations.drop('target').apply(lambda x: f"{abs(x):.2%}").values
    })
    corr_table['Orden'] = range(1, len(corr_table) + 1)
    print("\nVariables numéricas ordenadas por correlación con la conversión:")
    print(corr_table[['Orden', 'Variable', 'Correlación (%)']])




In [ ]:
# ============================================================================
# SECCIÓN 10: GUARDADO DEL MODELO Y SIMULACIÓN DE API
# ============================================================================

print("\n💾 GUARDANDO EL MODELO ENTRENADO")

# Guardamos el modelo usando pickle
with open('modelo_conversion_leads.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("✅ Modelo guardado como 'modelo_conversion_leads.pkl'")

# Simulación de cómo sería el despliegue del modelo como API
print("\n🚀 SIMULACIÓN DE DESPLIEGUE COMO API")

print("""
# Código para implementar la API con Flask (para producción)
from flask import Flask, request, jsonify
import pickle
import pandas as pd

app = Flask(__name__)

# Cargamos el modelo previamente entrenado
with open('modelo_conversion_leads.pkl', 'rb') as file:
    model = pickle.load(file)

@app.route('/predict', methods=['POST'])
def predict():
    # Recibimos los datos del lead
    data = request.json

    # Convertimos a DataFrame para que el modelo pueda procesarlo
    lead_df = pd.DataFrame([data])

    # Realizamos la predicción
    prediction = model.predict(lead_df)[0]
    probability = model.predict_proba(lead_df)[0][1]

    # Devolvemos el resultado
    return jsonify({
        'lead_id': data.get('Prospect_ID', 'unknown'),
        'conversion_prediction': int(prediction),
        'conversion_probability': float(probability),
        'recommendation': 'Priorizar contacto' if probability > 0.7 else 'Contacto regular'
    })

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)
""")

In [ ]:
# ============================================================================
# SECCIÓN 11: CONCLUSIONES Y PRÓXIMOS PASOS
# ============================================================================

print("\n📝 CONCLUSIONES DEL PROYECTO")
print("""
1. Hemos construido un pipeline completo de Machine Learning que permite predecir la
   probabilidad de conversión de leads en una empresa edtech.

2. El modelo logró un rendimiento de:
   - Precisión (Precision): Qué tan acertados somos cuando predecimos una conversión.
   - Recall: Qué porcentaje de los leads realmente interesados logramos identificar.
   - F1-Score: Balance entre precisión y recall.

3. Las características más importantes para predecir la conversión fueron identificadas,
   lo que permite entender mejor el comportamiento de los leads.

4. Este modelo permitirá al equipo comercial priorizar los leads con mayor probabilidad
   de conversión, optimizando recursos y mejorando la eficiencia.

5. Para la implementación en producción, se puede desplegar el modelo como una API
   que se integre con los sistemas CRM o de gestión de leads existentes.
""")


In [ ]:
# ============================================================================
# SECCIÓN 12: DEMO INTERACTIVA - PREDICCIÓN DE LEADS EN TIEMPO REAL
# ============================================================================

print("\n🔮 DEMO INTERACTIVA: PREDICCIÓN DE CONVERSIÓN DE LEADS")

# Primero aseguramos que el modelo está entrenado
try:
    # Si no existe la variable best_model en el entorno, mostrar un error
    if 'best_model' not in globals():
        print("❌ Error: No se encontró un modelo entrenado. Por favor, ejecuta primero la sección de entrenamiento.")
    else:
        from ipywidgets import widgets
        from IPython.display import display, clear_output, HTML

        # Obtenemos los nombres y tipos de características para crear el formulario
        feature_types = {}
        for col in X.columns:
            if col != 'Prospect_ID':  # No incluimos el ID en el formulario
                if col in numeric_features:
                    feature_types[col] = 'numeric'
                else:
                    feature_types[col] = 'categorical'

        # Función para crear el formulario dinámicamente basado en las columnas del dataset
        def create_lead_form():
            form_items = []

            # Título y descripción
            title = widgets.HTML(
                value="<h2 style='color: #4285f4; text-align: center;'>Predictor de Conversión de Leads</h2>"
                      "<p style='text-align: center;'>Ingrese los datos del lead para predecir su probabilidad de conversión</p>"
            )
            form_items.append(title)

            # Creamos widgets para cada característica
            input_widgets = {}

            # Características numéricas primero
            num_features_box = widgets.VBox([widgets.HTML("<h3>Datos Numéricos</h3>")])
            num_items = []

            for col in numeric_features:
                if col in X.columns and col != 'Prospect_ID':
                    # Para cada característica numérica, creamos un slider o campo numérico
                    min_val = max(0, X[col].min())
                    max_val = min(1000, X[col].max() * 1.5)  # Un poco más del máximo observado
                    default = X[col].median()

                    # Para valores pequeños, usamos FloatSlider
                    if max_val <= 100:
                        widget = widgets.FloatSlider(
                            value=default,
                            min=min_val,
                            max=max_val,
                            step=(max_val-min_val)/20,
                            description=f'{col}:',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(width='80%')
                        )
                    # Para valores más grandes, un campo numérico
                    else:
                        widget = widgets.FloatText(
                            value=default,
                            description=f'{col}:',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(width='80%')
                        )

                    input_widgets[col] = widget
                    num_items.append(widget)

            num_features_box.children = list(num_features_box.children) + num_items
            form_items.append(num_features_box)

            # Luego características categóricas
            cat_features_box = widgets.VBox([widgets.HTML("<h3>Datos Categóricos</h3>")])
            cat_items = []

            for col in categorical_features:
                if col in X.columns and col != 'Prospect_ID':
                    # Para cada característica categórica, creamos un dropdown
                    # Obtenemos valores únicos
                    unique_vals = X[col].dropna().unique().tolist()

                    # Limitamos a 20 valores para no sobrecargar la UI
                    if len(unique_vals) > 20:
                        unique_vals = unique_vals[:20]

                    # Creamos el dropdown
                    widget = widgets.Dropdown(
                        options=unique_vals,
                        value=unique_vals[0],
                        description=f'{col}:',
                        style={'description_width': 'initial'},
                        layout=widgets.Layout(width='80%')
                    )

                    input_widgets[col] = widget
                    cat_items.append(widget)

            cat_features_box.children = list(cat_features_box.children) + cat_items
            form_items.append(cat_features_box)

            # Botón de predicción
            predict_button = widgets.Button(
                description='Predecir Conversión',
                button_style='primary',
                layout=widgets.Layout(width='50%')
            )

            output = widgets.Output()

            # Función que se ejecuta al hacer clic en el botón
            def on_predict_button_clicked(b):
                # Limpiamos la salida anterior
                output.clear_output()

                # Recopilamos los datos del formulario
                new_lead = {}
                for col, widget in input_widgets.items():
                    new_lead[col] = widget.value

                # Mostramos el lead que se está evaluando
                with output:
                    print("Evaluando lead con las siguientes características:")
                    for col, val in new_lead.items():
                        print(f"  - {col}: {val}")

                    try:
                        # Convertimos a DataFrame
                        lead_df = pd.DataFrame([new_lead])

                        # Realizamos la predicción
                        prediction = best_model.predict(lead_df)[0]
                        probability = best_model.predict_proba(lead_df)[0][1]

                        # Mostramos los resultados con formato mejorado
                        result_color = "green" if prediction == 1 else "red"
                        result_text = "CONVERTIRÁ" if prediction == 1 else "NO CONVERTIRÁ"

                        # Determinamos la recomendación basada en la probabilidad
                        if probability > 0.8:
                            recommendation = "Prioridad ALTA: Contactar inmediatamente"
                            rec_color = "#1e8e3e"  # Verde oscuro
                        elif probability > 0.6:
                            recommendation = "Prioridad MEDIA: Contactar pronto"
                            rec_color = "#188038"  # Verde medio
                        elif probability > 0.4:
                            recommendation = "Prioridad NORMAL: Contactar en horario regular"
                            rec_color = "#fbbc04"  # Amarillo
                        elif probability > 0.2:
                            recommendation = "Prioridad BAJA: Contactar cuando haya disponibilidad"
                            rec_color = "#ea4335"  # Rojo claro
                        else:
                            recommendation = "Prioridad MUY BAJA: Considerar en campañas masivas"
                            rec_color = "#d93025"  # Rojo oscuro

                        # Mostramos el resultado con HTML para mejor formato
                        display(HTML(f"""
                        <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin-top: 20px;">
                            <h3 style="text-align: center; margin-bottom: 20px;">Resultado de la Predicción</h3>
                            <div style="display: flex; justify-content: center; margin-bottom: 15px;">
                                <div style="background-color: {result_color}; color: white; padding: 10px 20px; border-radius: 30px; font-size: 18px; font-weight: bold;">
                                    {result_text}
                                </div>
                            </div>
                            <div style="text-align: center; margin-bottom: 20px;">
                                <p style="font-size: 16px;">Probabilidad de conversión: <b>{probability:.2%}</b></p>
                                <div style="background-color: #e0e0e0; height: 20px; width: 100%; border-radius: 10px; overflow: hidden; margin-top: 10px;">
                                    <div style="background-color: {result_color}; height: 100%; width: {probability*100}%; border-radius: 10px;"></div>
                                </div>
                            </div>
                            <div style="background-color: {rec_color}; color: white; padding: 15px; border-radius: 8px; text-align: center; margin-top: 15px;">
                                <p style="margin: 0; font-weight: bold; font-size: 16px;">{recommendation}</p>
                            </div>
                        </div>
                        """))

                    except Exception as e:
                        print(f"❌ Error al realizar la predicción: {e}")

            predict_button.on_click(on_predict_button_clicked)

            # Centrar el botón
            button_container = widgets.HBox([widgets.Label(""), predict_button, widgets.Label("")],
                                          layout=widgets.Layout(justify_content='center', margin='20px 0px'))

            form_items.append(button_container)
            form_items.append(output)

            # Creamos el formulario completo
            form = widgets.VBox(form_items)
            return form

        # Mostramos el formulario
        print("Cargando formulario de predicción...")
        lead_form = create_lead_form()
        display(lead_form)

except Exception as e:
    print(f"❌ Error al crear el formulario interactivo: {e}")
    print("Para usar esta funcionalidad, asegúrate de tener instalado ipywidgets:")
    print("!pip install ipywidgets")
    print("Y ejecutar las secciones anteriores del notebook para entrenar el modelo.")